# Teaching GPT2 a sense of humor

This is simple experimental notebook for fine-tuning pretrained GPT2 model on jokes dataset. Let's see if it can learn to crack some jokes on it's own. 

For this purpose I will use pretrained models from huggingface [transformers repository](https://github.com/huggingface/transformers).

In [1]:
!pip install transformers

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

I1108 12:13:18.196740 140670293935872 file_utils.py:32] TensorFlow version 2.0.0 available.
I1108 12:13:18.197178 140670293935872 file_utils.py:39] PyTorch version 1.3.0 available.
I1108 12:13:18.505256 140670293935872 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


I will be using [GPT2LMHeadModel()](https://github.com/huggingface/transformers/blob/079bfb32fba4f2b39d344ca7af88d79a3ff27c7c/transformers/modeling_gpt2.py#L472) class, which is modified [GPT2Model()](https://github.com/huggingface/transformers/blob/079bfb32fba4f2b39d344ca7af88d79a3ff27c7c/transformers/modeling_gpt2.py#L320) for language modeling.

The only difference is added linear layer, which will transform the output embedding into logits (embedding size -> vocabulary size) which will be used for predicting the output words.

First I will do a small GPT2 model experiment to see if I have understood everything correctly and I can generate some text using the model.

[GPT2Tokenizer()](https://github.com/huggingface/transformers/blob/master/transformers/tokenization_gpt2.py#L106) is a byte-code pair encoder which will transform input text input into input tokens.

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [3]:
# Function to first select topN tokens from the probability list and then based on the N word distribution
# select random token ID
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

### First I prepare and tokenize the text which the model should start with continue itself. Then I run the model X iterations to add one token to the list in each iteration. 

I will start with the Morpheus famous monologe: 


***"The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth...***

In [4]:
cur_ids = torch.tensor(tokenizer.encode(" The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. ")).unsqueeze(0).long().to(device)

model.eval()
with torch.no_grad():
    
    for i in range(250):
        outputs = model(cur_ids, labels=cur_ids)
        loss, logits = outputs[:2]
        softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
        next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=5) #Randomly(from the given probability distribution) choose the next word from the top n words
        cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    print(output_text)

The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It's a world that you cannot see because it is so much greater than you.

The Matrix is the greatest invention of our time. It's an idea that was created in a single moment by a single person, the man who created the world, in one single moment. It's an idea that is now so pervasive that it has been translated into almost every culture, every language, every language of the world. You don't need to be a computer scientist in the 21st century to see that it has a profound effect. It's been the most profound invention ever made. It's been the most profound revolution of our time. It's the greatest invention since the invention of the steam engine.

The Matrix has ch

### For safety will save this masterpiece in a notebook cell

"The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth.

The Matrix is a world in which you can see and control everything, from your thoughts to the thoughts of your friends, your family members, the people you see in your dreams. It is a world that is constantly moving forward. You are a pawn, a puppet, a machine. The Matrix will always move forward, even if you can never see it.

In order to see the Matrix, you must understand it. You must understand what you are seeing. You must understand that you are not seeing what you want to see.

The Matrix is a machine that is always moving forward. The Matrix moves faster than any man or machine can move. The Matrix is a living being. It is the Matrix. It can never be destroyed, even if you can only see it.

You must understand that you are a part of this machine, and you have to learn to live with it and love it.

You must love this machine, but you must love it with all your heart as well, and you must love it with all your life, because it has a future for you.

The only way for you and your friends and your family and the world to live with the Matrix, is to "

### Not bad. The model works and now we are ready to teach the GPT2 some sense of humor.

### The first step is to find and prepare the data.

In [7]:
import os
import json

In [8]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json

class JokesDataset(Dataset):
    def __init__(self, jokes_dataset_path = 'jokes_data/'):
        super().__init__()

        reddit_jokes_path = os.path.join(jokes_dataset_path, 'reddit_jokes.json')

        with open(reddit_jokes_path) as f:
            data = json.load(f)

        self.joke_list = []
        self.end_of_text_token = "<|endoftext|>"

        for idx, joke_json in enumerate(data):
            joke_str = f"{self.end_of_text_token}START:{joke_json['title']} {joke_json['body']}{self.end_of_text_token}"
            self.joke_list.append(joke_str)
            
    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]


In [9]:
dataset = JokesDataset()
joke_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [7]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 1e-5
WARMUP_STEPS = 10000
MAX_SEQ_LEN = 550
from transformers import AdamW, WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [8]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
joke_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):
        
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
#             print(f"START!")
#             print(f"joke_tens sentence len {joke_tens.size()[1]}")
#             print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            continue
            
        else:
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
                
            else:
#                 print(f"ADDING joke_tens to tmp_jokes_tens")
#                 print(f"old tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
#                 print(f"joke_tens sentence len {joke_tens.size()[1]}")
#                 print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                
                continue
                
#         print(f"WORKING")
#         print(f"Working with {work_jokes_tens.size()[1]}")
#         print(f"New sentence len {joke_tens.size()[1]}")
#         print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        joke_count = joke_count + 1
        if joke_count == BATCH_SIZE:
            joke_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 1000:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

EPOCH 0 started==============================
sum loss 3182.91064453125
sum loss 2926.15185546875
sum loss 2849.66162109375
sum loss 2800.439208984375
sum loss 2777.589111328125
sum loss 2752.003173828125
sum loss 2734.627197265625
sum loss 2733.685791015625
sum loss 2696.652587890625
sum loss 2676.2119140625
sum loss 2675.95458984375
sum loss 2692.838134765625
sum loss 2688.138671875
sum loss 2679.025390625
sum loss 2694.001953125
sum loss 2671.19775390625
sum loss 2691.6298828125
sum loss 2681.33251953125
sum loss 2688.416748046875
sum loss 2699.18310546875
sum loss 2689.937255859375
sum loss 2692.32080078125
sum loss 2690.530517578125
sum loss 2699.981201171875
EPOCH 1 started==============================
sum loss 2686.610595703125
sum loss 2652.81884765625
sum loss 2678.482177734375
sum loss 2671.541015625
sum loss 2664.136962890625
sum loss 2673.405029296875
sum loss 2673.74072265625
sum loss 2658.427490234375
sum loss 2667.083984375
sum loss 2672.016357421875
sum loss 2666.85449

In [9]:
model.eval()
with torch.no_grad():
    
    for joke_idx in range(500):

        cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START:")).unsqueeze(0).to(device)
        
        for i in range(250):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            if i < 2:
                n = 15
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                break
            
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(f"JOKE NR {joke_idx}: {output_text} \n")

JOKE NR 0: <|endoftext|>START:Why do the Jews have such bad teeth? Because they are so bad at eating<|endoftext|> 

JOKE NR 1: <|endoftext|>START:Why don't you see the word 'bisexual' on a barstool? Because they're all gay.<|endoftext|> 

JOKE NR 2: <|endoftext|>START:My girlfriend is so fat......that she can't walk.<|endoftext|> 

JOKE NR 3: <|endoftext|>START:A priest, a rabbi, and two nuns are sitting in a bar... The priest asks, "What do you think about the fact that we are all going to hell?" The rabbi says, "I think it is a good thing we are all going to hell, because it is the only place that will be happy." The nun says, "It is a bad thing that we are all going to hell, because it is the only place that will be happy." The rabbi says, "I think it is a good thing, because we are all going to hell." The priest says, "What do you think about the fact that we are all going to hell?" The rabbi says, "I think it is a good thing we are all going to hell, because it is the only place t

JOKE NR 18: <|endoftext|>START:What is a pirate's favorite type of music? Pirate's music.  

(I'm sure you can guess what it is)<|endoftext|> 

JOKE NR 19: <|endoftext|>START:Why do you call a woman with a small penis a "small dick?" Because she's a little girl.<|endoftext|> 

JOKE NR 20: <|endoftext|>START:My wife is a bit......a bit of a bitch. She always says, "You're not my husband, I'm your sister".

So I said to myself, "I'm not your sister, I'm your husband!"<|endoftext|> 

JOKE NR 21: <|endoftext|>START:How do you know if you're a pedophile? You don't have to look at the picture.<|endoftext|> 

JOKE NR 22: <|endoftext|>START:A girl walks into a bar... She asks for a drink.
The bartender says "you're a girl."
The girl says "yeah, but why are you so drunk?"
The bartender says, "I'm not drunk. I'm a bartender." 
The girl says "I'm sorry, but why are you so drunk?"
The bartender says "I'm a girl."<|endoftext|> 

JOKE NR 23: <|endoftext|>START:A man and a woman are sitting at the ba

JOKE NR 35: <|endoftext|>START:I'm going to be the first person to die on a plane. I'm going to be the first person to die on a plane.<|endoftext|> 

JOKE NR 36: <|endoftext|>START:Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Because it was hungry.

Why did the dog cross the road? Beca

JOKE NR 50: <|endoftext|>START:Two people are sitting in a bar... One says to the second: "You have to tell me what's the difference between me and my wife?" The other replies: "Well, you know, I'm not married to my wife. We're both single."

"Well, that's not the difference, is it?"

"No, no. I'm not married to anyone."

"Oh, I see... That's not the same either, is it?"

"No, no. I'm not married to anyone."

"Well, then, what's the difference between you and your wife?"

"Well, you know, I'm not married to my wife either."

"Oh, I see... That's not the same either, is it?"

The other man says: "Well, I don't know, I guess it's just that I don't have a wife, and I don't want one either."

The bartender says: "Well, that's not the same either. You're not married to anyone either."

The second guy replies: "Well, that's not the same either, but you're not married to anyone either 

JOKE NR 51: <|endoftext|>START:How do you get a man to go on a date without being a total bitch? Give him a

JOKE NR 64: <|endoftext|>START:What's the difference between a black man and a black hole? The first one's black, the second one's black hole.<|endoftext|> 

JOKE NR 65: <|endoftext|>START:A man walks into a doctor's office......and asks the doctor if he can take a blood sample. 

The doctor replies, "I'm afraid I don't have any blood on me, but I'll give you a sample anyway."

"Okay. I'm going to have to go home and get some underwear, but I'm going to be late for work. What do you suggest I wear?" 

The man agrees, and he goes home to get his underwear. 

The doctor comes back a few minutes later and says, "Well, I'm afraid I don't have any panties on me. But you know what I'll do?" 

The man says, "I'll have to get some panties, but I'm going to be late for work anyway, so I'm going to have to wear a bra."

"Okay. I'm going to have to get some panties, and I'm going to be late for work. What do you suggest I wear?" 
The man agrees, and he goes home to get his panties. 

The doctor c

...

JOKE NR 413: <|endoftext|>START:The Man Who Was a Man and a Woman... The man who was a man and a woman were married. The man loved his wife and was very happy. He loved her, and he wanted to be happy. One day, the wife went out for a walk with her husband, and she saw a beautiful woman walking by the side of the road. She stopped and asked, “How are you doing today?” 

The man replied, “I am very happy, I am a man and a wife. I have been married for 25 years.” 

The woman then asked, “So what is your name, and how did you get married?”

The man replied, “I was born a man, and I was raised a woman, but I have always loved my wife. I have been with her for 25 years, and I love her very much. She is a beautiful woman, she is beautiful, and I have never been happier than I am now. I am happy, I am a happy man, and I have never felt happier than I am now. I am very happy, I am a very happy woman, and I have always loved me very much 

JOKE NR 414: <|endoftext|>START:The only thing worse tha

JOKE NR 427: <|endoftext|>START:A man walks into a doctor's office...... and says:

"Doctor, I have some problems with my penis. I've had it for a long time and it's still getting bigger and bigger, so I've been trying to get it out."

The doctor says:

"I'm sorry to hear that, it's very painful. You'll need some surgery to get rid of it, and you'll need some time."

The man agrees to the surgery. The doctor says:

"Now, you'll need to go to the local hospital, where they have a big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, big, 

JOKE NR 428: <|endoftext|>START:I've got a new job... A guy is a sales rep for a major corporation. He's always looking for new opportunities to make money. He's always trying to find a way to make m

JOKE NR 442: <|endoftext|>START:A man is driving down the highway......when he sees a sign that reads, "Stop for a drink."

The man pulls over and asks the driver to stop. The driver says, "Sorry, I'm not a drinker." The man says, "I'm a car driver." The driver says, "I'm not a car driver."

The man asks, "Why are you not a drinker?"

The driver says, "Because I'm a car driver."

The man says, "I'm a car driver, too."

The driver says, "I'm a car driver, too."

The man says, "Why are you not a drinker?"

The driver says, "Because I'm not a drinker.<|endoftext|> 

JOKE NR 443: <|endoftext|>START:When a man is in love with a woman... He goes into her bedroom and asks for a kiss. 
The woman is very surprised and asks him what he wants to do. 

He says, "I'm gonna go into the bathroom and get a drink of water and a glass of wine.  I'll come back in a minute." 

She then asks him, "What do you mean by a drink of water?" 

He replies, "I want a glass of wine."<|endoftext|> 

JOKE NR 444: <|e

JOKE NR 465: <|endoftext|>START:Three women are in a bar. The bartender asks if they'd like some drinks.

The first woman says she's a bit drunk.

The bartender says, "You've been here before, haven't you?"

The second woman says, "I'm not drunk, I'm just a little drunk."

The bartender says, "Well then, I'm going to have to make a change." He takes a seat next to the first woman and says,

"I've been drinking a lot of wine and beer lately. If I'm going to have to change the drinks, I think you should have a glass of wine and a glass of beer with it."

The second woman says, "I don't drink wine, I just like a good beer. I'll have the beer."

The third woman says, "Well, if I'm going to have a glass of wine and a glass of beer with it, then I'm going to have to go to the bar. If I'm going to have to go to the bar, I think I'm going to have to change my drink."

The bartender says, "I'm going to have to make a change. I 

JOKE NR 466: <|endoftext|>START:Why was the chicken cross? Because

JOKE NR 486: <|endoftext|>START:I got a new job as a bartender at an Italian restaurant. I have to be honest, I don't really enjoy my job, but I'm glad I got the opportunity. I'm not sure what the job entails, but I'm happy to be a part of it.<|endoftext|> 

JOKE NR 487: <|endoftext|>START:How do you get a man to do a thing? Ask him to do it.<|endoftext|> 

JOKE NR 488: <|endoftext|>START:Why can't you see a woman with no arms? She doesn't know how to use a hand.<|endoftext|> 

JOKE NR 489: <|endoftext|>START:A man and a girl are having sex......and suddenly a huge explosion comes from the sky.  The girl is screaming, "Help me! Help me!  What is that?"  The man says, "I don't know, I don't know, it's just a cloud."<|endoftext|> 

JOKE NR 490: <|endoftext|>START:How to get a man to go to the toilet A man walks in to a bar and sees a sign that says: "No men allowed". The man goes to the door, but the barman says no men allowed, so the man goes to the bathroom. The barman says "You're not

In [7]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 5e-5
WARMUP_STEPS = 10000
MAX_SEQ_LEN = 550
from transformers import AdamW, WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [8]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
joke_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):
        
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
#             print(f"START!")
#             print(f"joke_tens sentence len {joke_tens.size()[1]}")
#             print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            continue
            
        else:
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
                
            else:
#                 print(f"ADDING joke_tens to tmp_jokes_tens")
#                 print(f"old tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
#                 print(f"joke_tens sentence len {joke_tens.size()[1]}")
#                 print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                
                continue
                
#         print(f"WORKING")
#         print(f"Working with {work_jokes_tens.size()[1]}")
#         print(f"New sentence len {joke_tens.size()[1]}")
#         print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        joke_count = joke_count + 1
        if joke_count == BATCH_SIZE:
            joke_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 1000:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

EPOCH 0 started==============================
sum loss 3031.1455078125
sum loss 2826.515380859375
sum loss 2752.375244140625
sum loss 2728.57666015625
sum loss 2719.010498046875
sum loss 2692.240966796875
sum loss 2688.224609375
sum loss 2641.161865234375
sum loss 2655.967529296875
sum loss 2625.18212890625
sum loss 2639.4189453125
sum loss 2628.59228515625
sum loss 2631.25390625
sum loss 2637.0400390625
sum loss 2640.52392578125
sum loss 2623.533935546875
sum loss 2618.590087890625
sum loss 2640.354248046875
sum loss 2643.2607421875
sum loss 2645.875732421875
sum loss 2632.548095703125
sum loss 2621.15673828125
sum loss 2644.020263671875
sum loss 2636.111572265625
EPOCH 1 started==============================
sum loss 2616.936279296875
sum loss 2578.070068359375
sum loss 2583.287109375
sum loss 2559.488525390625
sum loss 2581.91650390625
sum loss 2577.4755859375
sum loss 2582.931640625
sum loss 2568.342529296875
sum loss 2586.42431640625
sum loss 2595.800048828125
sum loss 2566.083496

In [9]:
model.eval()
with torch.no_grad():
    
    for joke_idx in range(500):

        cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START:")).unsqueeze(0).to(device)
        
        for i in range(250):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            if i < 2:
                n = 15
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                break
            
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(f"JOKE NR {joke_idx}: {output_text} \n")

JOKE NR 0: <|endoftext|>START:A dog is walking through the woods... And a bear is running towards it.  The dog sees that bear, and runs away.

The dog sees the bear, but it is too late. The bear has already killed him.

The dog sees the bear again and runs away.  The bear sees that dog again and runs towards him.

The dog sees the bear and runs away.  The bear sees that dog and runs towards it.

The dog sees the bear, but it is too late. The bear is already dead.

The dog sees the bear, but it is too late. The bear is already dead.

The dog sees the bear, but it is too late. The bear is already dead.

The dog sees the bear, but it is too late. The bear is already dead.

The bear sees the dog, but it is too late. The bear is already dead.

The bear sees the dog, and the dog runs away, but the bear sees that dog again and runs towards it.

The dog sees the bear, and the bear sees the dog.

The bear sees the dog, and he runs away from the bear 

JOKE NR 1: <|endoftext|>START:How do you kn

JOKE NR 24: <|endoftext|>START:What do you call an Irishman who has no legs? A noleg.<|endoftext|> 

JOKE NR 25: <|endoftext|>START:Why did the chicken cross the road? Because he had to get to the other side.<|endoftext|> 

JOKE NR 26: <|endoftext|>START:Did you hear about the guy with no arms and no legs? He was just a little bit off<|endoftext|> 

JOKE NR 27: <|endoftext|>START:A guy goes to the doctor... A guy goes to the doctor for a prostate exam. The doc says, "I can't find anything wrong with your prostate."

"I'm sorry," the guy says, "but I'm afraid I have a rare genetic disorder."

"What is it?" the doc asks.

"It is a hereditary disease," the guy says.

"What is the condition?" the doctor asks.

"It's a rare disease," the guy responds. "It only affects the men who have the most money."

The doctor says, "You have to stop masturbating."

"I'm not masturbating, I'm just trying to find out how much money I have."

"You have to stop jerking off."

"It's not working," the guy say

JOKE NR 43: <|endoftext|>START:The two most important things in life are your parents. And your dog.<|endoftext|> 

JOKE NR 44: <|endoftext|>START:Two guys walk into a bar... Two guys walk into a bar. The first one says "I'm going to kill the bartender and make him my wife." The second guy replies with "I don't have a wife, but I do have a dick."<|endoftext|> 

JOKE NR 45: <|endoftext|>START:A blonde, brunette and blonde are walking down the street... A blond, brunette and blonde are all walking down the street when they see a man running away.

The brunette says "I'm going to catch him!"

So, the blonde goes after him, but she misses the guy.

The brunette says "Oh, I'm so sorry! I'm sorry!"

The blonde says "Oh, I'm so sorry! I'm so sorry! I'm so sorry!"

The brunette says "I'm so sorry!"

The blonde says "Oh, I'm so sorry!"

Then the brunette says to the blonde "I'm sorry, I'm so sorry! I'm going to catch him!"

So, she runs after him.

The blonde says "Oh, I'm so sorry! I'm so sorr

JOKE NR 59: <|endoftext|>START:I was in the shower the other day and the shower door opened and the water poured in. I thought it was the end of the shower.<|endoftext|> 

JOKE NR 60: <|endoftext|>START:What do you call a Mexican who has a fetish? A fetish. 




I'm Mexican.<|endoftext|> 

JOKE NR 61: <|endoftext|>START:I like my coffee like I like my slaves Free<|endoftext|> 

JOKE NR 62: <|endoftext|>START:What's the difference between a black guy and a white guy? You don't need a car to get to the store.<|endoftext|> 

JOKE NR 63: <|endoftext|>START:A black man walks in a bar......and asks, "Hey, what's that?"

"That's a beer," says the bartender

"That's a beer?" asks the bartender.

"That's a beer?" asks the black man.

"That's a beer," says the bartender.

"That's a beer?" says the black man,

"That's a beer?" says the bartender.

The black man says, "That's a beer."

"That's a beer," replies the bartender.

"That's a beer," says the black man,

"That's a beer," replies the barte

JOKE NR 79: <|endoftext|>START:What happens when the Pope dies? The Vatican will be renamed to the Pope of the new Pope.<|endoftext|> 

JOKE NR 80: <|endoftext|>START:Two guys are walking down the street......and they come upon an old house. They knock on the door and the old man opens it and says: "I'm sorry sir, I can only allow one person in this house, but you can have as many as you want."

So, one guy goes in and says: "I want a million dollars."

The other guy replies: "I don't have a million dollars, but I have a million ideas."

The first guy goes: "I want a mansion. It will have everything you want. It will have everything you need, and it will cost you nothing. It will be the most expensive house you've ever owned."

The other guy replies: "I don't want a mansion, I want a million bucks."

So, they go to the next house and the old man opens the door and says "You're welcome to have as many as you want."

So, one guy goes in and says "I want a million bucks."

The other guy r

...

JOKE NR 221: <|endoftext|>START:Did you know that the first person in the world to use a condom was a virgin? That's why it's called "the virgin condom."<|endoftext|> 

JOKE NR 222: <|endoftext|>START:What does a black man say? "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

An Indian says "Fuck you, you're black!"

An Arab says "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

An Indian says "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

An Indian says "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

An Indian says "Fuck you, you're black!"




A black guy says, "Fuck you, you're black!"

A black guy says, "Fuck you, you're black!"

An Indian says, "Fuck you, you're black!"

A black guy says "Fuck you, you're black!"

An Asian says, "Fuck you, you're black!"


An Indian says, "Fuck you, you're 

JOKE NR 223: <

JOKE NR 238: <|endoftext|>START:A priest, a lawyer, and a doctor are in the middle of a fight. They are all in the middle of a fight and the priest, the lawyer, and the doctor are all in the middle of a fight. They are all in the middle of a fight and they are all in the middle of a fight. The lawyer, who has been fighting the fight for the longest time, finally gets tired and says, "I'm tired."

The doctor says, "You're not tired."

The lawyer says, "I'm tired."

And finally, the priest gets tired. He says, "I'm tired."

The lawyer, who has been fighting all day, finally gets tired. He says, "You're not tired."

The doctor says, "You're not tired."

The priest says, "I am tired."

The lawyer, who has been fighting all night, finally gets tired. He says, "You're not tired."

The doctor says, "You're not tired."

Finally, the priest says, "I'm tired."

The lawyer says, "You're not tired."

The priest says, "I am 

JOKE NR 239: <|endoftext|>START:Did you hear about the man that was arres

JOKE NR 265: <|endoftext|>START:A woman is walking through the forest... A woman is walking through the forest when she comes across a bear.

She says to the animal "Hey! You're my bear!"
The animal replies, "I'm your bear, too!"

The woman says to the bear, "You're my bear!"

The bear replies, "I'm your bear, too."

The woman says to the bear, "You're my bear, too. I'm your bear, too."

The bear says, "You're my bear, too. I'm your bear, too."

The woman says to the bear, "You're my bear, too."

The bear says, "You're my bear, too. I'm your bear, too."

The woman says to the bear, "I'm your bear, too. I'm your bear, too."

The bear replies, "You're my bear, too. I'm your bear, too."

The woman says to the bear, "You're my bear, too. I'm your bear, too."

The bear says, "You're my bear, too. I'm your bear, 

JOKE NR 266: <|endoftext|>START:What's the best thing about a woman's breasts? They're so big, they can lift up your pants!<|endoftext|> 

JOKE NR 267: <|endoftext|>START:A little 

JOKE NR 287: <|endoftext|>START:What did the blind man say to the deaf man? "You're so ugly"<|endoftext|> 

JOKE NR 288: <|endoftext|>START:Two guys walk into a bar... One of them orders two beers and the other orders two shots. The bartender asks, "What's the occasion?" The first guy says, "I'm a vegetarian, so I'm not allowed to eat meat."

The bartender says, "You don't eat meat? That's a big no-no!"

The first guy says, "No, but I do like my beer cold. I like my beer cold."

The bartender says, "I'm sorry. You're not allowed to have beer cold."

The first guy says, "But I like my shots cold. I like my shots cold."

The bartender says, "You're not allowed to have your shots cold."

The first guy says, "But I do like my beer cold. I like my beer cold."

The bartender says, "You're not allowed to have your beer cold."

The first guy says, "But I do like my beer cold. I like my beer cold."

The second guy says, "I'm not allowed to have beer cold. I do like my beer cold."

The bartender

KeyboardInterrupt: 

In [10]:
BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 5e-5
WARMUP_STEPS = 10000
MAX_SEQ_LEN = 550
from transformers import AdamW, WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [9]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
joke_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):
        
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
#             print(f"START!")
#             print(f"joke_tens sentence len {joke_tens.size()[1]}")
#             print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            continue
            
        else:
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
                
            else:
#                 print(f"ADDING joke_tens to tmp_jokes_tens")
#                 print(f"old tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
#                 print(f"joke_tens sentence len {joke_tens.size()[1]}")
#                 print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                
                continue
                
#         print(f"WORKING")
#         print(f"Working with {work_jokes_tens.size()[1]}")
#         print(f"New sentence len {joke_tens.size()[1]}")
#         print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        joke_count = joke_count + 1
        if joke_count == BATCH_SIZE:
            joke_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 1000:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

EPOCH 0 started==============================
sum loss 3035.2158203125
sum loss 2830.087890625
sum loss 2756.093017578125
sum loss 2734.598388671875
sum loss 2700.84130859375
sum loss 2684.467529296875
sum loss 2688.87451171875
sum loss 2658.5556640625
sum loss 2646.299560546875
sum loss 2626.93505859375
sum loss 2644.5615234375
sum loss 2644.704833984375
sum loss 2634.573486328125
sum loss 2624.110107421875
sum loss 2634.079345703125
sum loss 2617.27783203125
sum loss 2628.5048828125
sum loss 2644.25439453125
sum loss 2642.25048828125
sum loss 2636.3154296875
sum loss 2654.2626953125
sum loss 2632.044921875
sum loss 2617.918701171875
sum loss 2631.875732421875


In [10]:
model.eval()
with torch.no_grad():
    
    for joke_idx in range(500):

        cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START:")).unsqueeze(0).to(device)
        
        for i in range(250):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            if i < 2:
                n = 15
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                break
            
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(f"JOKE NR {joke_idx}: {output_text} \n")

JOKE NR 0: <|endoftext|>START:Did you hear about the guy with a broken leg? It was his first day of work.<|endoftext|> 

JOKE NR 1: <|endoftext|>START:What do you call a group of people that's all addicted? A drug dealer's group<|endoftext|> 

JOKE NR 2: <|endoftext|>START:What happened to the blind woman? She got a new lamp.<|endoftext|> 

JOKE NR 3: <|endoftext|>START:What do you call a black guy who's a lawyer? A lawyer.

(I'll show myself out)<|endoftext|> 

JOKE NR 4: <|endoftext|>START:I was walking down the street when a guy stopped me and said "Hey, I've got a good story for you." I said "Wow, that sounds amazing!" and he said "Yeah, I know. I've been doing it for a long time." I said "Oh! I'm so sorry, but can you tell me what it is?" "Well," he replied "I've got it. I've been doing it for a long time. And I've been doing it for a long time. And I've been doing it for a long time." I said "Wow! I'm so sorry, but can you tell me what it is?" He said "Well," he replied "I've bee

JOKE NR 18: <|endoftext|>START:A man walks up to a girl and says, "Hey babe, I want to have sex with you." The girl looks at him and says, "I don't know, but I'm going to make it rain and I'm going to make it snow and I'm going to make it rain and I want to make it snow and I'm going to make it rain and I'm going to make it rain and I want to make it rain and I'm going to make it rain, but it's raining and I'm going to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I'm going to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I'm going to make it rain and I'm going to make it rain and I'm going to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I want to make it rain and I 

JOKE NR 19: <|endoftext|>START:

JOKE NR 40: <|endoftext|>START:An Irishman, a Scotsman, and a Welshman walk into a bar. The bartender says, “We’ve had some bad luck, and we’ll all be fine.” The Irishman says, “Well, I’m a bit of a gambler.” The bartender says, “You are a bit of a gambler, too?”<|endoftext|> 

JOKE NR 41: <|endoftext|>START:So a guy walks into a bar... and the barman says "I'm sorry, we don't serve alcohol here". The guy says "well, I'll have a glass of water and a glass of beer then"<|endoftext|> 

JOKE NR 42: <|endoftext|>START:Why did the chicken cross the road? Because he heard the farmer's son say, "I'll give you $50 if you can make me a sandwich."<|endoftext|> 

JOKE NR 43: <|endoftext|>START:A man is walking down the street when he sees a sign that says: "A man walks into a bar, and sees a beautiful woman in a negligee, naked."

The man asks the bartender: "What's that?"

The bartender says: "A beautiful woman in a negligee, naked."

The man says: "Wow, that's incredible!"

"It was a beautiful 

JOKE NR 59: <|endoftext|>START:I saw a sign that said, "If your car breaks down, it's your fault." I thought, "That's a good way to put it."

I got a new car and I'm still driving the old one.<|endoftext|> 

JOKE NR 60: <|endoftext|>START:A man is walking down a street... When a woman stops him.  "Hey, you know, I've got a really nice dress for you. I'm going to get it ready."   

"I'm not going to get that dress for you. You're a fucking idiot."<|endoftext|> 

JOKE NR 61: <|endoftext|>START:If a woman is pregnant and she's wearing a bra, does that mean she is wearing a bra? If it's not, does that mean she is pregnant?<|endoftext|> 

JOKE NR 62: <|endoftext|>START:My friend said he was gonna be a pirate... So i told him to get a pirate hat.<|endoftext|> 

JOKE NR 63: <|endoftext|>START:A blonde, a brunette, and a redhead are sitting on a park bench when the blonde says, "Hey! I got some good news and bad news."

"What's the good news?" the brunette asked.

"Well," the blonde says "the 

JOKE NR 83: <|endoftext|>START:How many feminists does it take to change a lightbulb? One to screw in the bulb and the other to hold the bulb in her hand while she's screwing it in<|endoftext|> 

JOKE NR 84: <|endoftext|>START:A man is walking down the beach when a shark jumps on his leg. He screams and falls off the edge. The next minute the shark jumps back on. The man screams and falls back. The next minute another shark jumps on. The man screams and falls again. The next minute the shark jumps on again and again.

"I'm going to die," he says.<|endoftext|> 

JOKE NR 85: <|endoftext|>START:I found a new way of telling jokes about sex......it was the best sex joke I've ever heard!<|endoftext|> 

JOKE NR 86: <|endoftext|>START:My dad was in the hospital the night of the explosion......and he said, "I'm so glad I didn't get a heart attack."

"Well, that's great news," said the nurse. "But what's so wonderful is that I got the heart attack."<|endoftext|> 

JOKE NR 87: <|endoftext|>START:

...

JOKE NR 406: <|endoftext|>START:Two Irishmen walk up to an old woman... One says to her, "You know, I've always been a little jealous of you."
The old woman replies "Well, I've always been a little jealous of you, too, but I'm getting married."
The second Irishman says, "You've got to be kidding me. I've got a beautiful wife, a beautiful daughter, a gorgeous dog and I'm getting married."
The old woman replies, "Well, that's a big plus, too, but I'm getting a new house, a new car, and a brand new apartment."<|endoftext|> 

JOKE NR 407: <|endoftext|>START:My girlfriend asked me if I wanted a blow job... I said "No, but if you want me to suck your dick..."<|endoftext|> 

JOKE NR 408: <|endoftext|>START:I was at a party the other day and I saw a guy wearing his best dress and tie. I thought, "Wow, that's a great dress!" Then I saw him with his best tie and dress and thought, "Wow, that's great tie!" Then I saw him wearing his best tie and dress and thought, "Wow, that's great tie!" Then I 

JOKE NR 428: <|endoftext|>START:The wife of the husband of the wife of the wife of the wife of the wife of the wife of the wife of the widow of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the wife of the widow of the wife of the wife of the widow of the wife of the wife of the wife of the widow of the wife of the widow of the wife of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widower of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widower of the widower of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of the widow of

JOKE NR 447: <|endoftext|>START:What's the difference between a hooker and an elephant? An elephant can't walk for more than a couple of hours.<|endoftext|> 

JOKE NR 448: <|endoftext|>START:A man goes to the doctor... A man goes to the doctor and asks him what he's going through. The doctor says, "Well, you know, I'm a little old man and I'm having trouble sleeping. I'm going to take you to the doctor, and he'll give you a prescription for something that will help you sleep."

So the man goes to the doctor and asks him the same thing. "Well," says the doctor, "I've been having a lot of trouble sleeping lately, too. I'll take you to the doctor and see him. He'll give you a prescription for something that will help you sleep."

So the man goes to the doctor and the doctor asks, "So, what do you want me to prescribe you for?" The man replies, "I've got some bad news and some bad news, too. I'm going to take you to the doctor and he'll give you a prescription for something that will help 

JOKE NR 470: <|endoftext|>START:Why couldn't the chicken go to the zoo? He was too big for the chicken.<|endoftext|> 

JOKE NR 471: <|endoftext|>START:What do you do when a black man goes to the doctor? You take a shit.<|endoftext|> 

JOKE NR 472: <|endoftext|>START:I was going to make a joke about the difference between the average man and the average woman. But I realized I'm not the average woman.<|endoftext|> 

JOKE NR 473: <|endoftext|>START:A joke I heard from a friend A guy walks into a bar. He's a big guy with a huge erection, so he's got to be the most attractive man in the bar. The bartender tells him, ‘Hey! I’ve got a really nice, sexy blonde, who is a gorgeous blonde. She’ll be the perfect girlfriend for you, and she’s also very pretty and beautiful,’ the bartender says. ‘Wow! That sounds great,’ says the guy. He goes to the bar and sits down, takes a seat, and the bartender asks him, ‘Hey, how about we get a drink?’ ‘Sure,’ says the man. ‘I’ll give her a drink.’ ‘Okay,’ sa

JOKE NR 494: <|endoftext|>START:I found out my girlfriend's dad is a pedophile......so I called him and asked, "Hey, Dad, are you fucking kidding me? I'm 14 and I'm still a virgin. What's going on?"

His reply was "Oh, I'm so sorry. I'm a pedophile, too. You see, when I was your age my dad would have sex with me, but I didn't have sex until I was 14, and then he'd have sex with me again. Then, when he was your age I'd have sex with him again. Then he'd have sex with me and again, and then he'd have sex with me and again, and he'd have sex with me and again. Then when he was your age I'd have sex with him again. And he would have sex with me and again. And then he'd have sex with me and again and again and again and again and again and then he would have sex with me and again, and then he'd have sex with me and again and again and again and then he'd have sex with me and again and again and again and again and then he'd have sex and again and again and then he would have sex and 

JOKE 

In [11]:
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 3e-5
WARMUP_STEPS = 10000
MAX_SEQ_LEN = 400
from transformers import AdamW, WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [12]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
seq_count = 0
tot_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):
        
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
#             print(f"START!")
#             print(f"joke_tens sentence len {joke_tens.size()[1]}")
#             print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            continue
            
        else:
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
                
            else:
#                 print(f"ADDING joke_tens to tmp_jokes_tens")
#                 print(f"old tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
#                 print(f"joke_tens sentence len {joke_tens.size()[1]}")
#                 print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
                
                continue
                
#         print(f"WORKING")
#         print(f"Working with {work_jokes_tens.size()[1]}")
#         print(f"New sentence len {joke_tens.size()[1]}")
#         print(f"New tmp_jokes_tens len {tmp_jokes_tens.size()[1]}")
            
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        seq_count = seq_count + 1
        tot_seq_count = tot_seq_count + 1
        
        if seq_count == BATCH_SIZE:
            seq_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if tot_seq_count % 1000 == 0:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0
            

EPOCH 0 started==============================
sum loss 3436.03076171875
sum loss 3271.525390625
sum loss 3117.43603515625
sum loss 3020.50732421875
sum loss 2940.70654296875
sum loss 2915.05078125
sum loss 2895.248779296875
sum loss 2849.1494140625
sum loss 2863.0771484375
sum loss 2827.261474609375
sum loss 2824.12109375
sum loss 2795.527587890625
sum loss 2803.104248046875
sum loss 2802.185791015625
sum loss 2786.28515625
sum loss 2778.56982421875
sum loss 2762.30615234375
sum loss 2770.957763671875
sum loss 2754.240478515625
sum loss 2747.343994140625
sum loss 2726.651611328125
sum loss 2729.69189453125
sum loss 2744.18212890625
sum loss 2724.10400390625
sum loss 2714.15283203125
sum loss 2711.803955078125
sum loss 2703.7119140625
sum loss 2704.89208984375
sum loss 2697.152099609375
sum loss 2689.19189453125
sum loss 2671.143798828125
sum loss 2677.786376953125
EPOCH 1 started==============================
sum loss 2645.66064453125
sum loss 2622.15966796875
sum loss 2662.0419921875


In [13]:
model.eval()
with torch.no_grad():
    
    for joke_idx in range(500):

        cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START:")).unsqueeze(0).to(device)
        
        for i in range(250):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            if i < 2:
                n = 15
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                break
            
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(f"JOKE NR {joke_idx}: {output_text} \n")

JOKE NR 0: <|endoftext|>START:What's the hardest to get off your face? The elevator. 

I'll show myself out.<|endoftext|> 

JOKE NR 1: <|endoftext|>START:A man is driving through the countryside when he sees a farmer in a field... He pulls over and asks the farmer, "Excuse me, but I noticed you're not wearing a seatbelt. What's the matter?" The farmer says, "I was just driving past the field and saw a cow grazing on a fence. It scared the bejesus out of me!" So the man pulls over to the side of the road and asks the farmer if he's okay. The farmer says "I'm fine, I was just driving past a field and saw a cow grazing on a fence. I thought it was a cow. I drove over to check and sure enough, it was a bull." The man says, "That's amazing, I was driving by the field when I saw a cow with her hind legs raised up in a sign of pride. I was so excited, I drove over to check and sure enough, it was a bull." The man says, "That's amazing, I was driving by the field when I saw a cow with its hind

JOKE NR 17: <|endoftext|>START:How do you get a girl pregnant? You take the tampon out of the box.<|endoftext|> 

JOKE NR 18: <|endoftext|>START:A man is walking through the forest when he sees an owl perched on a branch. The owl looks at him and says, 'I'm going to eat you.' 

'What's the harm?' the man asks. 

'I'm an American,' the owl replies. 'I can't eat a Canadian, so I'm going to eat you.' 

The man is taken aback, but he doesn't say anything and continues walking. 

A few minutes later, a rabbit is perched on a branch. 

'What's the harm?' the man asks the rabbit. 

'I'm an American,' the rabbit says. 'I can't eat a Canadian, so I'm going to eat you.' 

The rabbit is taken aback, but he doesn't say anything and continues walking. 

Finally, a bear is on the branch. 

'What's the matter with you, man?' the bear asks. 

The man replies, 'I'm an American, and I'm going to eat you.' 

'Why not?' the bear asks, 'I'm a Canadian too.' 

The man replies, 'I 

JOKE NR 19: <|endoftext|>

JOKE NR 36: <|endoftext|>START:Two men are walking through the woods when they come across a bear. One of their friend asks, "Hey, what are you doing?"

The other man replies, "We're hunters."<|endoftext|> 

JOKE NR 37: <|endoftext|>START:Why would you never trust a man with a broken pencil? He's a pencil thief.<|endoftext|> 

JOKE NR 38: <|endoftext|>START:A man is walking through a forest... A man is walking through the woods when he comes across a bear. He approaches it, and after a few minutes of staring at the bear, the bear stops.

"You're not going to eat me?" the bear asked.

The man replies, "No, you're not going to eat me. I have a gun in my pocket and a knife with me, and I don't want you to get any closer."

The bear then proceeds to climb up on top of the man, and the bear starts to eat the man. The bear then looks at the man, and says, "I'm going to give you the same punishment I gave the bear. You have to eat me."

The man, shocked by this, says, "What punishment? I didn

JOKE NR 55: <|endoftext|>START:What did the pirate say to the pirate? "I am the best!"<|endoftext|> 

JOKE NR 56: <|endoftext|>START:A blonde, a redhead and a brunette are all sitting in a bar. A blonde, a redhead and a brunette are all sitting in a bar. The blonde looks over to the brunette and says "You know what, you know I can take the whole bar with me. I'm a brunette and I can take a blonde with me."

The brunette says "I'll take you with me, too. I'm also a brunette and I can take a redhead." The redhead looks over at the blonde, and says "You know what, that's a no brainer. I'll take you with me. I'm a redhead too. I can take a blonde too." The brunette looks at him and says "Well, I'm a blonde too, but I can take a brunette with me."

The blonde looks at her friend and says "Well, you're right. If you want to go with me, go ahead. I'm a blonde, too. I'm a blonde too."

The blonde looks back and says "Well, I'll be damned. If I'm going to be a blond, I want to be a blond blonde

JOKE NR 78: <|endoftext|>START:The Perfect Son. A: I have the perfect son. 
B: Does he smoke? 
A: No, he doesn't. 
B: Does he drink whiskey? 
A: No, he doesn't. 
B: Does he ever come home late? 
A: No, he doesn't. 
B: I guess you really do have the perfect son. How old is he? 
A: He will be six months old next Wednesday.<|endoftext|> 

JOKE NR 79: <|endoftext|>START:An American, a Russian and a German are on a plane... An American, a Russian and a German are on a plane. The plane is going down and the pilot is yelling, “We’re going down. We can't get back up. We’ve only got two parachutes.” The American jumps and lands. He jumps again and lands. The German jumps and the plane crashes to the ground.

The pilot yells, “We have parachutes. Get up.” The parachutes are taken off and the pilot yells, “Get up, you stupid bastards! You’ve ruined our lives and I’ll nail your fucking feet to the floor!”<|endoftext|> 

JOKE NR 80: <|endoftext|>START:The man who invented the vacuum cleaner......wa

...

JOKE NR 415: <|endoftext|>START:Did you hear about the guy who was arrested for stealing a car? He was charged with car theft<|endoftext|> 

JOKE NR 416: <|endoftext|>START:A man goes to the doctor for a check up. Doctor : I have some bad news.  You have to stop masturbating.
Man : What's wrong?
Doctor : Well I'm afraid you've got cancer.
Man : What? What is it?
Doctor : Your testicles are cancerous.
Man : Wow that's awful, what can I do?
Doctor : You can stop masturbating and you can come back in a week.

Man comes back the week after and the Doctor says "Well, you have to stop masturbating, I'm afraid you have cancer.  What's the problem?"
Man : Doctor, I don't have cancer.
Doctor : Well I'm afraid you have AIDS.
Man : What?! What's AIDS?
Doctor : Your testicles are cancerous.
Man : Wow that's awful, what can I do?
Doctor : Go to the doctor, he'll give you the cure.<|endoftext|> 

JOKE NR 417: <|endoftext|>START:Why aren't there any black jokes on this subreddit? I'm not sure, I've h

JOKE NR 436: <|endoftext|>START:A woman is walking through a park......and she comes across a dog. The dog is obviously a pit bull, and the woman thinks to herself, "Wow, I wish I had this dog."

The woman goes up to the dog and says, "I'm going to give you a bone."

The dog says to her, "You're a real bitch, you're not supposed to give a bone to a pit bull. I'm going to bite you in the ass."

The woman is shocked. "Oh my god, I thought you said you were a pit bull."

The woman says, "Yeah. I'm going to bite your balls."

The dog says, "No, no. I'm going to bite your tits. You're not supposed to give a bone to a pit bull."

The woman says, "Oh my god, I thought you said you were a pit bull. I'm going to bite your tits!"

The dog says, "No, no. I'm gonna bite your tits!"

The woman says, "No, no, no, no, no, no, no, no, no, no, no, no, 

JOKE NR 437: <|endoftext|>START:A guy walks into a bar......and sits down at the bar. He orders three beers and starts drinking them. He finishes them 

JOKE NR 455: <|endoftext|>START:What did the pirate say to the pirate captain when they were in trouble? "I'll be a good boy and let's go back."<|endoftext|> 

JOKE NR 456: <|endoftext|>START:So, I went to the doctor......and he asked me if I had ever had sex before.

"No" I replied.

"Well, you have to stop masturbating."

"What?" I said. "I'm over here."

"You have to stop touching yourself."

"Oh, I'm over here too."

"You have to stop masturbating."

"What?" I asked.

"You have to stop touching yourself."

I said, "But, I'm over here."

The Doctor looked at me and said, "You have to stop masturbating, or I'll call the cops."

I said, "But, I'm over here too."

"Well, then, you have to stop touching yourself."

I looked at the doctor, then back to him and said, "But I'm over here too."

"You have to stop masturbating."

"What?"

"You have to stop masturbating."

"I can't stop masturbating, I'm over here too."

"You have to stop masturbating."

"What?
 

JOKE NR 457: <|endoftext|>STA

JOKE NR 472: <|endoftext|>START:An elderly couple were having dinner... An elderly couple were having dinner and they were having some conversation. 

The wife asks, "What's your favorite letter in the alphabet?" 

The husband replies, "I love the letter 'C' because it's the hardest letter in the world to pronounce."

The wife asks, "What's your favorite letter in the alphabet?"

The husband replies, "I love the letter 'D', because it's the most common letter in the world. 

The wife says, "Well, what's your favorite letter in the alphabet?"

The husband says, "The letter 'F'. 

The wife asks, "Well what's your favorite letter in the alphabet?" 

The husband responds, "The letter 'G', because it's the most common letter in the world."

The wife asks, "Well what's your favorite letter in the alphabet?"

The husband says, "The letter 'H'. 

The wife says, "What the hell, what's your favorite letter in the alphabet?"

The husband says, "The letter 'I'. 
The wife asks, "Well what is 

JOKE

JOKE NR 488: <|endoftext|>START:A man goes to the hospital to have his appendix removed. The doctor asks the man, "How long have you been having these?"

The man says, "About a month and a half."

"What happened?" asks the doctor.

"Well, my wife and I had a little fight and I had to go to the hospital. I was in the operating room, and I was in the middle of the operation, and I heard my wife screaming and crying, 'I'm pregnant! I'm pregnant!'"

The doctor asks, "What was she doing?"

The man says, "She was trying to get me to stop screaming."<|endoftext|> 

JOKE NR 489: <|endoftext|>START:A man goes into a bar... A man goes into a bar. He sits at a table and orders a beer.

The bartender says, "You know what this is? A double entendre."

The man looks at him and says, "No, no. This is a double entendre."

"A double entendre? Why not a triple entendre?"

"No, no, this is a double entendre."

The man says, "What do you mean, a triple entendre?"

"No, no, this is a triple entendre."

The

In [14]:
torch.save(model.state_dict(), os.path.join("gpt2_medium_joker.pt"))

In [21]:
def gen_sentences(start_str = "", n = 5):
    model.eval()
    with torch.no_grad():

        for joke_idx in range(n):

            cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START: " + start_str)).unsqueeze(0).to(device)

            for i in range(250):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
                if i < 2:
                    n = 15
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    break

            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)
            print(f"JOKE NR {joke_idx}: {output_text} \n")